## 1. Setting Up this Notebook
We will start with a few imports.

In [2]:
from llama_stack_client import LlamaStackClient
import os
import time
import requests
from dotenv import load_dotenv
import json
import urllib.parse
from datetime import datetime, timezone, timedelta

load_dotenv()

# === Enhanced Loki Query Function with Improved JSON Parsing ===
def query_loki_logs(namespace: str, container_name: str, hours: int = 1) -> str:
    """Query logs from a namespace and container in LokiStack with enhanced JSON parsing."""
    try:
        print(f"🔍 Querying Loki logs for namespace: {namespace}, container: {container_name}, hours: {hours}")
        
        # Updated token (use your actual token)
        token = os.getenv("TOKEN")
        
        # Calculate time range in ISO format
        now = datetime.now(timezone.utc)
        start_time = now - timedelta(hours=hours)
        
        # Format timestamps as ISO 8601 strings
        start_iso = start_time.strftime("%Y-%m-%dT%H:%M:%SZ")
        end_iso = now.strftime("%Y-%m-%dT%H:%M:%SZ")
        
        # Base URL for Loki
        base_url = os.getenv("LOKI_BASE_URL")
        url = f"{base_url}/api/logs/v1/application/loki/api/v1/query_range"
        
        # Use the working LogQL query pattern - search by container name in pod names
        # Since container names are often part of pod names, we'll use regex matching
        logql_query = f'{{kubernetes_namespace_name="{namespace}",kubernetes_pod_name=~".*{container_name}.*"}}'
        
        headers = {
            "Authorization": f"Bearer {token}",
            "Content-Type": "application/json",
            "Accept": "application/json"
        }
        
        params = {
            "query": logql_query,
            "start": start_iso,
            "end": end_iso,
            "limit": 5000
        }
        
        print(f"📡 Making request to: {url}")
        print(f"📋 Query: {logql_query}")
        print(f"🕒 Time range: {start_iso} to {end_iso}")
        
        res = requests.get(
            url, 
            params=params, 
            headers=headers, 
            timeout=30
        )
        
        print(f"📊 Response Status: {res.status_code}")
        
        if res.status_code == 200:
            try:
                response_data = res.json()
                
                # Check if we got data
                if response_data.get("status") == "success":
                    parsed_logs = []
                    results = response_data.get("data", {}).get("result", [])
                    
                    for result in results:
                        for entry in result.get("values", []):
                            if len(entry) >= 2:
                                # entry[0] is timestamp (nanoseconds), entry[1] is log message
                                timestamp_ns = entry[0]
                                raw_log = entry[1]
                                
                                # Convert timestamp from nanoseconds to human-readable format
                                try:
                                    timestamp_seconds = int(timestamp_ns) / 1_000_000_000
                                    formatted_timestamp = datetime.fromtimestamp(timestamp_seconds, tz=timezone.utc).strftime("%Y-%m-%d %H:%M:%S UTC")
                                except (ValueError, TypeError):
                                    formatted_timestamp = str(timestamp_ns)
                                
                                # Parse the log message (similar to jq logic)
                                parsed_message = parse_log_message(raw_log)
                                
                                # Format output: timestamp + parsed message
                                log_line = f"{formatted_timestamp} {parsed_message}"
                                parsed_logs.append(log_line)
                                print(log_line)
                    
                    if parsed_logs:
                        log_output = "\n".join(parsed_logs)
                        print(f"✅ Successfully retrieved and parsed {len(parsed_logs)} log entries")
                        return f"Found {len(parsed_logs)} parsed log entries for container '{container_name}' in namespace '{namespace}':\n\n{log_output}"
                    else:
                        print("📭 No log entries found in results")
                        return f"❌ No logs found for container '{container_name}' in namespace '{namespace}' for the last {hours} hour(s)"
                else:
                    error_msg = f"❌ Loki returned error status: {response_data.get('error', 'Unknown error')}"
                    print(error_msg)
                    return error_msg
                    
            except json.JSONDecodeError as je:
                error_msg = f"❌ JSON decode error: {je}"
                print(error_msg)
                return error_msg
                
        else:
            error_msg = f"❌ HTTP Error {res.status_code}: {res.text}"
            print(error_msg)
            return error_msg
        
    except requests.exceptions.RequestException as e:
        error_msg = f"❌ Request failed: {str(e)}"
        print(error_msg)
        return error_msg
    except Exception as e:
        error_msg = f"❌ Unexpected error querying Loki: {str(e)}"
        print(error_msg)
        return error_msg


def parse_log_message(raw_log: str) -> str:
    """
    Parse log message similar to jq logic:
    Try to parse as JSON and extract msg/message/log fields, 
    otherwise return the raw log.
    """
    try:
        # Try to parse as JSON
        log_json = json.loads(raw_log)
        
        # Extract the message field (similar to jq: .msg // .message // .log // .)
        if isinstance(log_json, dict):
            # Priority order: msg -> message -> log -> entire object
            message = (
                log_json.get('msg') or 
                log_json.get('message') or 
                log_json.get('log') or 
                str(log_json)
            )
            return str(message)
        else:
            # If it's not a dict, return as string
            return str(log_json)
            
    except (json.JSONDecodeError, TypeError):
        # If it's not JSON, return the raw log
        return raw_log


def query_loki_logs_raw_format(namespace: str, container_name: str, hours: int = 1) -> str:
    """
    Alternative version that returns logs in a more raw format,
    exactly mimicking the jq output format.
    """
    try:
        print(f"🔍 Querying Loki logs (raw format) for namespace: {namespace}, container: {container_name}, hours: {hours}")
        
        token = os.getenv("TOKEN")
        now = datetime.now(timezone.utc)
        start_time = now - timedelta(hours=hours)
        
        start_iso = start_time.strftime("%Y-%m-%dT%H:%M:%SZ")
        end_iso = now.strftime("%Y-%m-%dT%H:%M:%SZ")
        
        base_url = os.getenv("LOKI_BASE_URL")
        url = f"{base_url}/api/logs/v1/application/loki/api/v1/query_range"
        
        logql_query = f'{{kubernetes_namespace_name="{namespace}",kubernetes_pod_name=~".*{container_name}.*"}}'
        
        headers = {
            "Authorization": f"Bearer {token}",
            "Content-Type": "application/json",
            "Accept": "application/json"
        }
        
        params = {
            "query": logql_query,
            "start": start_iso,
            "end": end_iso,
            "limit": 5000
        }
        
        res = requests.get(url, params=params, headers=headers, timeout=30)
        
        if res.status_code == 200:
            response_data = res.json()
            
            if response_data.get("status") == "success":
                raw_output_lines = []
                results = response_data.get("data", {}).get("result", [])
                
                for result in results:
                    for entry in result.get("values", []):
                        if len(entry) >= 2:
                            timestamp_ns = entry[0]
                            raw_log = entry[1]
                            
                            # Convert timestamp to string (keeping the number format like jq)
                            timestamp_str = str(int(timestamp_ns) // 1_000_000_000)  # Convert to seconds
                            
                            # Parse the log message using the same logic
                            parsed_message = parse_log_message(raw_log)
                            
                            # Format exactly like jq output: "timestamp message"
                            line = f"{timestamp_str} {parsed_message}"
                            raw_output_lines.append(line)
                
                if raw_output_lines:
                    return "\n".join(raw_output_lines)
                else:
                    return f"No logs found for container '{container_name}' in namespace '{namespace}'"
            else:
                return f"Loki error: {response_data.get('error', 'Unknown error')}"
        else:
            return f"HTTP Error {res.status_code}: {res.text}"
            
    except Exception as e:
        return f"Error querying Loki: {str(e)}"


# === Fixed LlamaStack 0.2.6 Agent Creation ===
def create_llamastack_agent():
    """Create LlamaStack agent using the correct 0.2.6 API."""
    base_url = os.getenv("REMOTE_BASE_URL")
    if not base_url:
        print("❌ REMOTE_BASE_URL environment variable not set")
        return None, None
        
    client = LlamaStackClient(base_url=base_url)
    
    try:
        print("🤖 Creating agent using LlamaStack 0.2.6 API...")
        
        # Create agent configuration - corrected for 0.2.6
        agent_config = {
            "model": "granite32-8b",
            "instructions": """You are a helpful assistant that retrieves logs from Kubernetes containers using Loki.

When users ask for logs:
1. Use the query_loki_logs tool to retrieve actual logs from containers
2. Present the logs in a readable format with timestamps and parsed messages
3. If no logs are found, suggest checking container/namespace names
4. Provide insights about what you see in the logs if relevant

The logs are returned in a parsed format showing timestamp and the actual log message content.
Always use the tool when log data is requested rather than giving general explanations.""",
            "tools": [{
                "type": "function",
                "function": {
                    "name": "query_loki_logs",
                    "description": "Query logs from Kubernetes containers using Loki with enhanced JSON parsing to extract clean log messages",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "namespace": {
                                "type": "string",
                                "description": "Kubernetes namespace name"
                            },
                            "container_name": {
                                "type": "string", 
                                "description": "Container name to query logs from"
                            },
                            "hours": {
                                "type": "integer",
                                "description": "Number of hours back to query logs (default: 1)",
                                "default": 1,
                                "minimum": 1,
                                "maximum": 24
                            }
                        },
                        "required": ["namespace", "container_name"]
                    }
                }
            }],
            "sampling_params": {
                "strategy": {"type": "greedy"},
                "max_tokens": 512,
                "temperature": 0.1
            }
        }
        
        # Create the agent - this returns AgentCreateResponse
        agent_response = client.agents.create(agent_config=agent_config)
        print(f"✅ Agent created successfully!")
        print(f"🆔 Agent ID: {agent_response.agent_id}")
        
        return client, agent_response.agent_id
        
    except Exception as e:
        print(f"💥 Agent creation failed: {e}")
        import traceback
        traceback.print_exc()
        return None, None


# === FIXED Session and Turn Management for 0.2.6 ===
def create_session(client, agent_id):
    """Create session using the correct 0.2.6 API structure."""
    try:
        print(f"💬 Creating session for agent {agent_id}...")
        
        session_name = f"loki-session-{int(time.time())}"
        
        # Try different approaches to create session
        session_response = None
        
        # Method 1: client.agents.session.create (most likely)
        try:
            print("🔄 Trying client.agents.session.create...")
            session_response = client.agents.session.create(
                agent_id=agent_id,
                session_name=session_name
            )
            print("✅ Method 1 worked!")
        except Exception as e1:
            print(f"❌ Method 1 failed: {e1}")
            
            # Method 2: Check what methods are available on session resource
            try:
                print("🔍 Checking available methods on session resource...")
                session_methods = [attr for attr in dir(client.agents.session) if not attr.startswith('_')]
                print(f"Session methods: {session_methods}")
                
                # Try common method names
                if hasattr(client.agents.session, 'create'):
                    print("🔄 Trying session.create...")
                    session_response = client.agents.session.create(
                        agent_id=agent_id,
                        session_name=session_name
                    )
                elif hasattr(client.agents.session, 'new'):
                    print("🔄 Trying session.new...")
                    session_response = client.agents.session.new(
                        agent_id=agent_id,
                        session_name=session_name
                    )
                elif hasattr(client.agents.session, 'start'):
                    print("🔄 Trying session.start...")
                    session_response = client.agents.session.start(
                        agent_id=agent_id,
                        session_name=session_name
                    )
                else:
                    raise Exception(f"No suitable method found in: {session_methods}")
                    
            except Exception as e2:
                print(f"❌ Method 2 failed: {e2}")
                
                # Method 3: Try direct POST if we have to
                try:
                    print("🔄 Trying alternative approach...")
                    # Check if there's a different way to access sessions
                    if hasattr(client, 'sessions'):
                        print("🔄 Trying client.sessions.create...")
                        session_response = client.sessions.create(
                            agent_id=agent_id,
                            session_name=session_name
                        )
                    else:
                        raise Exception("No sessions attribute found")
                        
                except Exception as e3:
                    print(f"❌ All methods failed: {e1}, {e2}, {e3}")
                    return None
        
        if session_response:
            # Extract session_id from response
            session_id = getattr(session_response, 'session_id', None)
            if not session_id:
                session_id = getattr(session_response, 'id', None)
            
            if session_id:
                print(f"✅ Session created: {session_id}")
                return session_id
            else:
                print(f"❌ Could not extract session_id from response: {session_response}")
                return None
        else:
            return None
        
    except Exception as e:
        print(f"❌ Session creation failed: {e}")
        import traceback
        traceback.print_exc()
        return None


def create_turn_streaming(client, agent_id, session_id, user_message):
    """Create turn using STREAMING - the only supported method."""
    try:
        print(f"💬 Creating STREAMING turn for session {session_id}...")
        
        # Use streaming response since non-streaming is not implemented
        turn_response = client.agents.turn.create(
            agent_id=agent_id,
            session_id=session_id,
            messages=[{
                "role": "user",
                "content": user_message
            }],
            stream=True  # Enable streaming
        )
        
        print(f"✅ Streaming turn created successfully")
        
        # Process streaming response
        collected_response = ""
        turn_id = None
        
        for chunk in turn_response:
            #print(f"🔄 Stream chunk: {chunk}")
            
            # Extract turn_id from first chunk
            if hasattr(chunk, 'turn_id') and not turn_id:
                turn_id = chunk.turn_id
               # print(f"🆔 Turn ID: {turn_id}")
            
            # Collect response content
            if hasattr(chunk, 'content'):
                collected_response += chunk.content
            elif hasattr(chunk, 'delta') and hasattr(chunk.delta, 'content'):
                collected_response += chunk.delta.content
            elif hasattr(chunk, 'message') and hasattr(chunk.message, 'content'):
                collected_response += chunk.message.content
        
        print(f"📝 Collected response: {collected_response[:200]}...")
        
        return {
            'turn_id': turn_id,
            'content': collected_response,
            'streaming_response': True
        }
        
    except Exception as e:
        print(f"❌ Streaming turn creation failed: {e}")
        import traceback
        traceback.print_exc()
        return None


def create_turn_with_streaming_response(client, agent_id, session_id, user_message):
    """Alternative streaming approach using with_streaming_response."""
    try:
        print(f"💬 Creating turn with streaming response...")
        
        # Use the streaming response wrapper
        with client.agents.turn.with_streaming_response.create(
            agent_id=agent_id,
            session_id=session_id,
            messages=[{
                "role": "user",
                "content": user_message
            }]
        ) as response:
            print(f"📡 Streaming response received")
            
            collected_content = ""
            turn_id = None
            
            for line in response.iter_lines():
                if line:
                    print(f"🔄 Line: {line}")
                    try:
                        data = json.loads(line)
                        
                        # Extract turn_id
                        if 'turn_id' in data and not turn_id:
                            turn_id = data['turn_id']
                        
                        # Extract content
                        if 'content' in data:
                            collected_content += data['content']
                        elif 'delta' in data and 'content' in data['delta']:
                            collected_content += data['delta']['content']
                            
                    except json.JSONDecodeError:
                        # Skip non-JSON lines
                        continue
            
            return {
                'turn_id': turn_id,
                'content': collected_content,
                'streaming_response': True
            }
        
    except Exception as e:
        print(f"❌ Streaming response turn failed: {e}")
        import traceback
        traceback.print_exc()
        return None


def try_alternative_agent_approaches(client, agent_id, session_id, user_message):
    """Try alternative approaches when standard agent turns don't work."""
    
    print("\n🔧 Trying alternative agent approaches...")
    
    # Approach 1: Direct inference API instead of agent turns
    try:
        print("🔄 Trying direct inference API...")
        
        if hasattr(client, 'inference'):
            # Prepare the message with tool context
            enhanced_message = f"""
You are a helpful assistant with access to a Loki log querying tool.

User request: {user_message}

Available tool:
- query_loki_logs(namespace: str, container_name: str, hours: int = 1) -> str

Please respond to the user's request about logs.
"""
            
            inference_response = client.inference.chat_completion(
                model="granite32-8b",
                messages=[{
                    "role": "user", 
                    "content": enhanced_message
                }],
                max_tokens=512,
                temperature=0.1
            )
            
            print(f"✅ Direct inference worked!")
            return {
                'content': inference_response.choices[0].message.content,
                'method': 'direct_inference'
            }
            
    except Exception as e:
        print(f"❌ Direct inference failed: {e}")
    
    # Approach 2: Direct chat completion
    try:
        print("🔄 Trying direct chat completion...")
        
        if hasattr(client, 'chat'):
            chat_response = client.chat.completions.create(
                model="granite32-8b",
                messages=[{
                    "role": "system",
                    "content": "You are a helpful assistant that can analyze log requests and guide users."
                }, {
                    "role": "user",
                    "content": user_message
                }],
                max_tokens=512,
                temperature=0.1
            )
            
            print(f"✅ Direct chat completion worked!")
            return {
                'content': chat_response.choices[0].message.content,
                'method': 'chat_completion'
            }
            
    except Exception as e:
        print(f"❌ Direct chat completion failed: {e}")
    
    return None


def get_agent_steps(client, agent_id, session_id, turn_id):
    """Get the steps/execution of the agent turn."""
    try:
        print(f"🔍 Getting steps for turn {turn_id}...")
        
        # Try different approaches to get steps
        steps_response = None
        
        # Method 1: client.agents.steps.list or similar
        try:
            print("🔄 Trying client.agents.steps...")
            
            # Check what methods are available on steps resource
            if hasattr(client.agents, 'steps'):
                steps_methods = [attr for attr in dir(client.agents.steps) if not attr.startswith('_')]
                print(f"Steps methods: {steps_methods}")
                
                # Try common method names
                if hasattr(client.agents.steps, 'list'):
                    print("🔄 Trying steps.list...")
                    steps_response = client.agents.steps.list(
                        agent_id=agent_id,
                        session_id=session_id,
                        turn_id=turn_id
                    )
                elif hasattr(client.agents.steps, 'get'):
                    print("🔄 Trying steps.get...")
                    steps_response = client.agents.steps.get(
                        agent_id=agent_id,
                        session_id=session_id,
                        turn_id=turn_id
                    )
                elif hasattr(client.agents.steps, 'retrieve'):
                    print("🔄 Trying steps.retrieve...")
                    steps_response = client.agents.steps.retrieve(
                        agent_id=agent_id,
                        session_id=session_id,
                        turn_id=turn_id
                    )
                else:
                    # Try calling it directly with parameters
                    print("🔄 Trying direct call...")
                    steps_response = client.agents.steps(
                        agent_id=agent_id,
                        session_id=session_id,
                        turn_id=turn_id
                    )
            else:
                raise Exception("No steps attribute found")
                
        except Exception as e1:
            print(f"❌ Steps method failed: {e1}")
            return None
        
        return steps_response
        
    except Exception as e:
        print(f"❌ Getting steps failed: {e}")
        import traceback
        traceback.print_exc()
        return None


# === Process Steps and Execute Tool Calls ===
def process_agent_steps(steps_response):
    """Process the agent steps and execute tool calls."""
    if not steps_response:
        return "No steps received from agent"
    
    print(f"🔍 Processing steps response: {type(steps_response)}")
    
    # Handle different response types
    if hasattr(steps_response, 'steps'):
        print(f"📝 Processing {len(steps_response.steps)} steps...")
        
        for i, step in enumerate(steps_response.steps):
            print(f"Step {i+1}: {step}")
            print(f"Step type: {type(step)}")
            
            # Check if this step contains a tool call
            if hasattr(step, 'step_type'):
                print(f"🔧 Step type: {step.step_type}")
                
                if step.step_type == 'tool_execution' or 'tool' in str(step.step_type).lower():
                    print(f"🔧 Tool execution step found")
                    
                    # Look for tool calls in different possible attributes
                    tool_calls = None
                    if hasattr(step, 'tool_calls'):
                        tool_calls = step.tool_calls
                    elif hasattr(step, 'tool_call'):
                        tool_calls = [step.tool_call]
                    elif hasattr(step, 'function_call'):
                        tool_calls = [step.function_call]
                    
                    if tool_calls:
                        for tool_call in tool_calls:
                            print(f"🔍 Processing tool call: {tool_call}")
                            
                            # Extract function name and arguments
                            function_name = None
                            arguments = {}
                            
                            if hasattr(tool_call, 'function'):
                                function_name = tool_call.function.name
                                try:
                                    arguments = json.loads(tool_call.function.arguments)
                                except:
                                    arguments = tool_call.function.arguments
                            elif hasattr(tool_call, 'name'):
                                function_name = tool_call.name
                                arguments = getattr(tool_call, 'arguments', {})
                            
                            if function_name == 'query_loki_logs':
                                print(f"🔍 Executing Loki query with args: {arguments}")
                                result = query_loki_logs(**arguments)
                                print(f"📋 Loki result: {result[:200]}...")
                                return result
            
            # Check if step has content directly
            elif hasattr(step, 'content'):
                return step.content
    
    # If no tool calls found, return the raw response
    return str(steps_response)


# === Improved Chat Wrapper for 0.2.6 ===
class LokiChatWrapper:
    """Improved chat wrapper for LlamaStack 0.2.6 with enhanced log parsing."""
    
    def __init__(self, client, agent_id):
        self.client = client
        self.agent_id = agent_id
        self.session_id = None
        
    def ensure_session(self):
        """Ensure we have a valid session."""
        if not self.session_id:
            self.session_id = create_session(self.client, self.agent_id)
        return self.session_id is not None
    
    def send_message(self, message: str):
        """Send a message and handle the response."""
        if not self.ensure_session():
            return "Could not create session"
        
        # Check if this is a Loki query request
        if self.is_loki_request(message):
            # Try agent first, fallback to direct parsing
            try:
                return self.send_to_agent(message)
            except Exception as e:
                print(f"❌ Agent failed: {e}, trying direct parsing...")
                return self.handle_loki_request_direct(message)
        
        # For non-Loki requests, send to agent
        return self.send_to_agent(message)
    
    def send_to_agent(self, message: str):
        """Send message to agent and process response."""
        try:
            # First try streaming turn
            turn_response = create_turn_streaming(self.client, self.agent_id, self.session_id, message)
            
            if not turn_response:
                # Try alternative streaming approach
                print("🔄 Trying alternative streaming approach...")
                turn_response = create_turn_with_streaming_response(self.client, self.agent_id, self.session_id, message)
            
            if not turn_response:
                # Try alternative agent approaches
                print("🔄 Trying alternative agent approaches...")
                alternative_response = try_alternative_agent_approaches(self.client, self.agent_id, self.session_id, message)
                if alternative_response:
                    return alternative_response['content']
            
            if turn_response and turn_response.get('streaming_response'):
                # For streaming responses, we already have the content
                content = turn_response.get('content', '')
                
                # Check if the agent mentioned tool usage or if we should parse directly
                if 'query_loki_logs' in content or self.is_loki_request(message):
                    # Try to extract parameters and call Loki directly
                    print("🔍 Agent response suggests Loki query, trying direct call...")
                    direct_result = self.handle_loki_request_direct(message)
                    
                    # Combine agent response with actual results
                    if direct_result and "Found" in direct_result:
                        return f"Agent response: {content}\n\nParsed log results:\n{direct_result}"
                    else:
                        return content
                else:
                    return content
            
            # Original non-streaming approach (keeping for compatibility)
            if turn_response and not turn_response.get('streaming_response'):
                turn_id = getattr(turn_response, 'turn_id', getattr(turn_response, 'id', None))
                if turn_id:
                    print(f"✅ Turn created with ID: {turn_id}")
                    
                    # Wait a moment for processing
                    time.sleep(1)
                    
                    # Get steps
                    steps_response = get_agent_steps(self.client, self.agent_id, self.session_id, turn_id)
                    if steps_response:
                        return process_agent_steps(steps_response)
                
            return "Could not process agent response"
            
        except Exception as e:
            print(f"❌ Send to agent failed: {e}")
            import traceback
            traceback.print_exc()
            return f"Error sending to agent: {e}"
    
    def is_loki_request(self, message: str) -> bool:
        """Check if the message is requesting Loki logs."""
        keywords = ['logs', 'log', 'container', 'namespace', 'loki', 'kubernetes', 'k8s']
        return any(keyword in message.lower() for keyword in keywords)
    
    def handle_loki_request_direct(self, message: str):
        """Parse the message and execute Loki query directly."""
        # Simple parsing - you can make this more sophisticated
        words = message.lower().split()
        namespace = None
        container_name = None
        hours = 1
        
        # Look for namespace and container
        for i, word in enumerate(words):
            if word in ['namespace', 'ns'] and i + 1 < len(words):
                namespace = words[i + 1]
            elif word in ['container', 'container-name', 'service'] and i + 1 < len(words):
                container_name = words[i + 1]
            elif word in ['hours', 'hour'] and i > 0:
                try:
                    hours = int(words[i - 1])
                except:
                    hours = 1
        
        # Try to extract from common patterns
        if not namespace or not container_name:
            # Look for patterns like "container-name in namespace-name"
            if ' in ' in message:
                parts = message.split(' in ')
                if len(parts) >= 2:
                    container_part = parts[0].strip()
                    namespace_part = parts[1].strip()
                    
                    # Extract container name (last word of first part)
                    if not container_name:
                        container_words = container_part.split()
                        if container_words:
                            container_name = container_words[-1]
                    
                    # Extract namespace (first word of second part)
                    if not namespace:
                        ns_words = namespace_part.split()
                        if ns_words:
                            namespace = ns_words[0]
        
        # Special handling for customer-validation-service
        if not container_name and 'customer-validation-service' in message:
            container_name = 'customer-validation-service'
        
        if namespace and container_name:
            print(f"🔍 Parsed request: namespace='{namespace}', container='{container_name}', hours={hours}")
            return query_loki_logs(namespace, container_name, hours)
        else:
            return f"Please specify both namespace and container name. Example: 'Get logs from container customer-validation-service in namespace customer-onboarding'\n\nParsed: namespace='{namespace}', container='{container_name}'"


# === Alternative Direct API Approach ===
def try_direct_api_calls(client, agent_id):
    """Try direct API calls using different patterns."""
    print("\n🔧 Trying direct API patterns...")
    
    session_name = f"direct-session-{int(time.time())}"
    
    # Pattern 1: Resource.method pattern
    patterns_to_try = [
        # Session creation patterns
        ("Session Create Pattern 1", lambda: client.agents.session.create(agent_id=agent_id, session_name=session_name)),
        ("Session Create Pattern 2", lambda: client.agents.session.new(agent_id=agent_id, session_name=session_name)),
        ("Session Create Pattern 3", lambda: client.sessions.create(agent_id=agent_id, session_name=session_name) if hasattr(client, 'sessions') else None),
    ]
    
    for pattern_name, pattern_func in patterns_to_try:
        try:
            print(f"🔄 Trying {pattern_name}...")
            result = pattern_func()
            if result:
                print(f"✅ {pattern_name} worked! Result: {result}")
                return result
        except Exception as e:
            print(f"❌ {pattern_name} failed: {e}")
    
    return None


# === Main execution ===
def main():
    print("="*60)
    print("🚀 LlamaStack 0.2.6 Agent with Enhanced Loki Integration")
    print("📋 Features: Enhanced JSON parsing similar to jq output")
    print("="*60)
    
    # Test the enhanced parsing functions first
    print("\n🧪 Testing enhanced log parsing functions...")
    
    # Test parse_log_message function
    test_json_log = '{"timestamp":"2024-01-01T12:00:00Z","level":"INFO","msg":"User login successful","user_id":12345}'
    test_plain_log = "2024-01-01 12:00:00 INFO: Database connection established"
    
    print(f"📝 Test JSON log parsing:")
    print(f"   Input: {test_json_log}")
    print(f"   Output: {parse_log_message(test_json_log)}")
    
    print(f"📝 Test plain log parsing:")
    print(f"   Input: {test_plain_log}")
    print(f"   Output: {parse_log_message(test_plain_log)}")
    
    # Create agent
    print("\n1. Creating LlamaStack agent...")
    client, agent_id = create_llamastack_agent()
    
    if client and agent_id:
        print(f"✅ Agent created successfully! ID: {agent_id}")
        
        # Try direct API calls
        print("\n2. Testing direct API patterns...")
        direct_result = try_direct_api_calls(client, agent_id)
        
        # Test the new session creation
        print("\n3. Testing session creation...")
        session_id = create_session(client, agent_id)
        
        if session_id:
            print(f"✅ Session created successfully! ID: {session_id}")
            
            # Test streaming turn creation
            print("\n4. Testing STREAMING turn creation...")
            user_message = "Please get logs from the customer-validation-service container in the customer-onboarding namespace for the last 50  hour"
            
            # Try streaming approach
            turn_response = create_turn_streaming(client, agent_id, session_id, user_message)
            
            if turn_response:
                print(f"✅ Streaming turn created successfully!")
                #print(f"📝 Response content: {turn_response.get('content', 'No content')[:300]}...")
            else:
                print("❌ Streaming turn creation failed")
        
        # Test chat wrapper
        print("\n5. Testing enhanced chat wrapper...")
        chat = LokiChatWrapper(client, agent_id)
        
        test_messages = [
            "Get logs from container customer-validation-service in namespace customer-onboarding for last 100 hours",
            #"Show me logs from container web-server in production namespace for 1 hour"
        ]
        
        for i, msg in enumerate(test_messages, 1):
            print(f"\n🔄 Test message {i}: {msg}")
            result = chat.send_message(msg)
            print(f"📝 Result: {result[:500]}..." if len(str(result)) > 500 else f"📝 Result: {result}")
    
    else:
        print("❌ Could not create agent")
        print("\n🔧 Testing enhanced Loki function directly...")
        print("\n📋 Testing standard format:")
        result = query_loki_logs("customer-onboarding", "customer-validation-service", 1)
        print(f"📋 Standard result: {result}")
        
        print("\n📋 Testing raw format (jq-like):")
        raw_result = query_loki_logs_raw_format("customer-onboarding", "customer-validation-service", 1)
        print(f"📋 Raw result: {raw_result}")


if __name__ == "__main__":
    main()

INFO:httpx:HTTP Request: POST http://llamastack-server:8321/v1/agents "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://llamastack-server:8321/v1/agents/0e875231-d1b2-4bbe-ba52-e8c89ddabb4f/session "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://llamastack-server:8321/v1/agents/0e875231-d1b2-4bbe-ba52-e8c89ddabb4f/session "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://llamastack-server:8321/v1/agents/0e875231-d1b2-4bbe-ba52-e8c89ddabb4f/session/dab3e6cc-6a19-43d8-9a22-0da6dcf8abfc/turn "HTTP/1.1 200 OK"


🚀 LlamaStack 0.2.6 Agent with Enhanced Loki Integration
📋 Features: Enhanced JSON parsing similar to jq output

🧪 Testing enhanced log parsing functions...
📝 Test JSON log parsing:
   Input: {"timestamp":"2024-01-01T12:00:00Z","level":"INFO","msg":"User login successful","user_id":12345}
   Output: User login successful
📝 Test plain log parsing:
   Input: 2024-01-01 12:00:00 INFO: Database connection established
   Output: 2024-01-01 12:00:00 INFO: Database connection established

1. Creating LlamaStack agent...
🤖 Creating agent using LlamaStack 0.2.6 API...
✅ Agent created successfully!
🆔 Agent ID: 0e875231-d1b2-4bbe-ba52-e8c89ddabb4f
✅ Agent created successfully! ID: 0e875231-d1b2-4bbe-ba52-e8c89ddabb4f

2. Testing direct API patterns...

🔧 Trying direct API patterns...
🔄 Trying Session Create Pattern 1...
✅ Session Create Pattern 1 worked! Result: SessionCreateResponse(session_id='48874849-5e2f-4f21-a007-8aeae70682b2')

3. Testing session creation...
💬 Creating session for agent 0e8

INFO:httpx:HTTP Request: POST http://llamastack-server:8321/v1/agents/0e875231-d1b2-4bbe-ba52-e8c89ddabb4f/session "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://llamastack-server:8321/v1/agents/0e875231-d1b2-4bbe-ba52-e8c89ddabb4f/session/861115fc-dce7-4a8e-9bda-7a47dce1b417/turn "HTTP/1.1 200 OK"


📝 Collected response: ...
✅ Streaming turn created successfully!

5. Testing enhanced chat wrapper...

🔄 Test message 1: Get logs from container customer-validation-service in namespace customer-onboarding for last 100 hours
💬 Creating session for agent 0e875231-d1b2-4bbe-ba52-e8c89ddabb4f...
🔄 Trying client.agents.session.create...
✅ Method 1 worked!
✅ Session created: 861115fc-dce7-4a8e-9bda-7a47dce1b417
💬 Creating STREAMING turn for session 861115fc-dce7-4a8e-9bda-7a47dce1b417...
✅ Streaming turn created successfully
📝 Collected response: ...
🔍 Agent response suggests Loki query, trying direct call...
🔍 Parsed request: namespace='customer-onboarding', container='customer-validation-service', hours=100
🔍 Querying Loki logs for namespace: customer-onboarding, container: customer-validation-service, hours: 100
📡 Making request to: https://logging-loki-openshift-logging.apps.cluster-srb7z.srb7z.sandbox2014.opentlc.com/api/logs/v1/application/loki/api/v1/query_range
📋 Query: {kubernetes_